In [1]:
import numpy as np
import pandas as pd
import yfinance as yf


In [2]:
def clean(ticker):
    ydf = yf.download(ticker)
    ydf.to_csv(f'y{ticker}.csv')

    btc = pd.read_csv(f'y{ticker}.csv')
    btc = btc.drop([0,1])
    btc['Date'] = pd.to_datetime(btc['Price']).dt.date
    btc= btc.set_index('Date')
    btc = btc.drop(['Price'], axis = 1)
    btc = btc.reset_index()
    btc['Ticker'] = ticker
    return btc



In [3]:
btc = clean('BTC-USD')
eth = clean('ETH-USD')
dc = clean('DOGE-USD')
usdc = clean('USDC-USD')
bnb = clean('BNB-USD')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
#combine data
data = pd.concat([btc,eth,dc,usdc,bnb])
data['Date'] = pd.to_datetime(data.Date)
data = data.sort_values('Date')
data = data.query("Date >=  '2018-10-08'")

In [5]:
#rename and fix dtype
data = data.rename(columns = {'Date': 'date',
                              'Adj Close': 'adj_prc',
                              'Close': 'prc',
                              'High': 'high',
                              'Low': 'low',
                              'Open': 'open',
                              'Volume': 'vol',
                              "Ticker": 'tick'
                            })

data = data.apply(lambda x: x.astype(float) if x.dtype == 'object' and x.name != 'tick' else x)
data.set_index('date',inplace=True)
data.reset_index(inplace = True)
data.dtypes


date       datetime64[ns]
adj_prc           float64
prc               float64
high              float64
low               float64
open              float64
vol               float64
tick               object
dtype: object

In [6]:
#returns
data['ret'] = (data.adj_prc - data.open)/data.open

In [7]:
#momentum, one month rolling window
data['logret'] = np.log(1 + data['ret'])
data['mom'] = data.groupby('tick')['logret'].rolling(30,30).sum().reset_index(drop=True)
data['mom'] = data.groupby('tick')['mom'].shift(2)
data = data.dropna(subset='mom')

In [8]:
#dispersion
data['disp'] = (data.high - data.low)/data.open

In [9]:
#lag returns
data['lagret'] = data.groupby('tick')['ret'].shift(1)
data = data.dropna(subset='lagret')

In [10]:
data.to_csv('proj_data.csv')